In [ ]:
import datetime
import tweepy
import json
import time
import emoji
from tweepy.error import TweepError

access_token = "3249597226-fr2Cc6SnoQayvTnc6qCL7mkBDYAEIo1JxgSqJbJ"
access_token_secret = "6SSiNbfdjmLu6Nsh9vdoMEgMlgdl5sOLIBn7TkkfM8Cfg"
consumer_key = "Zp8BVvUHOJKnsH4pgS1Vtxrh1"
consumer_secret = "CnMsDDeR7Yu6KRAMOjPXdjA8Az31OAlHGl0s4YG66SCdbYzYrx"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
print(api)
original_tweet=0
filtered_count=0
reply_count=0
RC=0
def extract_tweets(poi,max_id,id_lis,original_tweet):
    try:
        #original_tweet=0
        filtered_count=0
        poi_tweet_file = poi + 'newtweets.json'
        end_encountered = 0
        any_tweet_found = True;
        while(original_tweet < 1800 and any_tweet_found ):
            #print(any_tweet_found)

            any_tweet_found = False
            #print(any_tweet_found)
            if max_id == 0:
                print("Default call")
                cur = tweepy.Cursor(api.user_timeline, screen_name=poi, tweet_mode='extended').items(2000)
            else:
                print("Max Id call")
                cur = tweepy.Cursor(api.user_timeline, screen_name=poi,max_id = max_id-1 , tweet_mode='extended').items(2000)
            for tweet in cur:
                 
                 max_id = tweet.id
               
                 #print(hasattr(tweet, 'retweeted_status'))
                 if not hasattr(tweet, 'retweeted_status') and not tweet.in_reply_to_status_id:
                    #print("changing the any_tweet_found to true at line 115")
                    any_tweet_found = True


                    if original_tweet >=1800:
                      break
                   
                    original_tweet+=1
                   
                    if tweet.created_at > min_date and tweet.created_at < max_date:
                        filtered_count+=1
                        id_lis.append(tweet.id)
                        print(tweet.id)
                    with open(poi_tweet_file, 'a+', encoding="utf8") as f:
                        obj = convertToObj(tweet)
                        print(obj["tweet_date"])

                        json.dump(obj, f, ensure_ascii=False)
    except TweepError as e:
        time.sleep(10)
        print(e)
        extract_tweets(poi,max_id,id_lis,)


def extract_replies(poi,id_list,replies_lis,replies_count_map,max_id):
    try:
        poi_replies_file = poi + 'newreplies.json'
        i =0;
        since_id = min(id_list)
        # max_id = None
        # temp_id = None
        total_tweets = 0
        last_max_id = max_id
        extraction_finished = 0;
        while (1):
            print("Fetching tweets for since id :"+str(since_id)+" max id is :"+str(max_id))
            if max_id == 0:
                cur = tweepy.Cursor(api.search, q='to:'+poi, since_id=since_id, count=1000,tweet_mode='extended').items(1000)
            else:
                cur = tweepy.Cursor(api.search, q='to:'+poi, since_id=since_id, max_id=max_id-1,count=1000,tweet_mode='extended').items(1000)
            for tweet in cur:
                max_id = tweet.id
                if tweet.in_reply_to_status_id in id_list:
                    total_tweets = total_tweets+1
                    if len(replies_count_map[tweet.in_reply_to_status_id]) <= 20:
                        replies_lis.append(tweet)
                        obj = convertToObj(tweet)
                        obj["poi_name"] = poi
                        obj["poi_id"] = obj["replied_to_user_id"]
                        replies_count_map[tweet.in_reply_to_status_id].append(obj)
                        print("Found reply for tweet :"+str(tweet.in_reply_to_status_id) +" ,reply with id : "+str(tweet.id)+" ,count is" + str(len(replies_count_map[tweet.in_reply_to_status_id])))
            if last_max_id == max_id or all(len(v) >= 20 for v in replies_count_map.values()):

                break
       
    except TweepError:
       time.sleep(10)
       extract_replies(poi,id_list,replies_lis,replies_count_map,max_id)

def round_to_hour(dt):
    dt_start_of_hour = dt.replace(minute=0, second=0, microsecond=0)
    dt_half_hour = dt.replace(minute=30, second=0, microsecond=0)

    if dt >= dt_half_hour:
        # round up
        dt = dt_start_of_hour + datetime.timedelta(hours=1)
    else:
        # round down
        dt = dt_start_of_hour

    return dt

def get_place_obj(place):
    objec = {}
    objec['country_code'] = place.country_code
    objec['coordinates'] = place.bounding_box.coordinates
    objec['name'] = place.name
    #print(objec)
    return objec

def convertToObj(tweet):
    obj={}
    obj["poi_name"] = tweet.user.screen_name
    obj["poi_id"] = tweet.user.id
    obj["verified"] = tweet.user.verified
    obj["country"] = "India"
    obj["replied_to_tweet_id"] = tweet.in_reply_to_status_id_str
    obj["replied_to_user_id"] = tweet.in_reply_to_user_id_str
    if tweet.in_reply_to_user_id:
        obj["reply_text"] = tweet.full_text
    obj["tweet_text"] = tweet.full_text
    obj["tweet_lang"] = tweet.lang
    obj["hashtags"] = tweet.entities["hashtags"]
    obj["mentions"] = tweet.entities["user_mentions"]
    obj["tweet_urls"] = tweet.entities["urls"]
    obj["retweeted"] = tweet.retweeted
    if not tweet.place is None:
        obj["tweet_loc"] = get_place_obj(tweet.place)
    obj["id"] = tweet.id
    obj["retweet_count"] = tweet.retweet_count
    obj["favorite_count"] = tweet.favorite_count
    obj["favorited"] = tweet.favorited
    obj["tweet_emoticons" ] = ''.join(c for c in tweet.full_text if c in emoji.UNICODE_EMOJI)
    obj["tweet_date"] = str(round_to_hour(tweet.created_at))
    return obj

min_date = datetime.datetime(2019, 9, 7)
max_date = datetime.datetime(2019, 9, 12)
poi = 'BDUTT'
poi_tweet_file = poi + 'newtweets.json'
poi_replies_file = poi + 'newreplies.json'
id_list = []
#id_list.append(1172466267502067714)
#tweets = api.get_status('1172461856952217600') # id_list is the list of tweet ids
#print("tweets are " + str(tweets))
extract_tweets(poi,0,id_list,0)
#obj = convertToObj(tweets)
time.sleep(1)
replies_list=[]
count_map = {}
print(len(id_list))
for id in id_list:
    count_map[id] = []
extract_replies(poi,id_list,replies_list,count_map,0)
with open(poi_replies_file, 'a+', encoding="utf8") as f:
    for id in id_list:
        for obj in count_map[id]:
             json.dump(obj, f, ensure_ascii=False)          


   
time.sleep(1)
print('Original count', original_tweet)
print('Filtered Replies count', filtered_count)
print('Tot Replies count', RC, len(replies_list), list)


Default call
2019-09-15 15:00:00
2019-09-15 12:00:00
2019-09-15 11:00:00
2019-09-15 08:00:00
2019-09-15 08:00:00
2019-09-15 07:00:00
2019-09-14 17:00:00
2019-09-14 13:00:00
2019-09-14 12:00:00
2019-09-14 09:00:00
2019-09-13 14:00:00
2019-09-13 03:00:00
2019-09-12 18:00:00
2019-09-12 09:00:00
2019-09-12 08:00:00
2019-09-12 04:00:00
2019-09-12 03:00:00
1171820366030766080
2019-09-11 16:00:00
1171792610773651457
2019-09-11 14:00:00
1171773802080088064
2019-09-11 13:00:00
1171693620941819904
2019-09-11 08:00:00
1171648150366539776
2019-09-11 05:00:00
1171470063112609793
2019-09-10 17:00:00
1171451584481394695
2019-09-10 16:00:00
1171359045908750337
2019-09-10 10:00:00
1171274550220185606
2019-09-10 04:00:00
1171273529339174913
2019-09-10 04:00:00
1171265121026330624
2019-09-10 04:00:00
1171131245213806592
2019-09-09 19:00:00
1171107671253995521
2019-09-09 17:00:00
1171098515344257024
2019-09-09 17:00:00
1171047503686504450
2019-09-09 13:00:00
1170958082546925568
2019-09-09 07:00:00
1170949

2019-07-11 16:00:00
2019-07-11 15:00:00
2019-07-11 15:00:00
2019-07-11 14:00:00
2019-07-10 18:00:00
2019-07-10 15:00:00
2019-07-10 14:00:00
2019-07-10 08:00:00
2019-07-09 11:00:00
2019-07-09 07:00:00
2019-07-08 15:00:00
2019-07-07 16:00:00
2019-07-07 16:00:00
2019-07-07 07:00:00
2019-07-07 05:00:00
2019-07-07 05:00:00
2019-07-06 12:00:00
2019-07-06 12:00:00
2019-07-06 11:00:00
2019-07-06 09:00:00
2019-07-06 08:00:00
2019-07-06 06:00:00
2019-07-05 09:00:00
2019-07-05 08:00:00
2019-07-05 08:00:00
2019-07-05 03:00:00
2019-07-04 15:00:00
2019-07-04 09:00:00
2019-07-04 09:00:00
2019-07-04 05:00:00
2019-07-04 05:00:00
2019-07-04 02:00:00
2019-07-03 17:00:00
2019-07-03 15:00:00
2019-07-03 11:00:00
2019-07-03 10:00:00
2019-07-03 10:00:00
2019-07-03 04:00:00
2019-07-03 04:00:00
2019-07-03 04:00:00
2019-07-02 11:00:00
2019-07-02 10:00:00
2019-07-02 10:00:00
2019-07-01 18:00:00
2019-07-01 15:00:00
2019-07-01 15:00:00
2019-07-01 06:00:00
2019-07-01 06:00:00
2019-07-01 06:00:00
2019-06-30 14:00:00


2019-05-10 07:00:00
2019-05-10 06:00:00
2019-05-10 06:00:00
2019-05-10 05:00:00
2019-05-09 17:00:00
2019-05-09 17:00:00
2019-05-09 15:00:00
2019-05-09 13:00:00
2019-05-09 12:00:00
2019-05-09 09:00:00
2019-05-09 09:00:00
2019-05-09 09:00:00
2019-05-09 08:00:00
2019-05-09 07:00:00
2019-05-09 07:00:00
2019-05-09 06:00:00
2019-05-09 05:00:00
2019-05-09 05:00:00
2019-05-09 04:00:00
2019-05-09 04:00:00
2019-05-09 04:00:00
2019-05-08 17:00:00
2019-05-08 14:00:00
2019-05-08 12:00:00
2019-05-08 11:00:00
2019-05-08 11:00:00
2019-05-08 09:00:00
2019-05-08 08:00:00
2019-05-08 08:00:00
2019-05-08 05:00:00
2019-05-08 05:00:00
2019-05-08 04:00:00
2019-05-08 03:00:00
2019-05-07 19:00:00
2019-05-07 18:00:00
2019-05-07 17:00:00
2019-05-07 16:00:00
2019-05-07 15:00:00
2019-05-07 15:00:00
2019-05-07 12:00:00
2019-05-07 09:00:00
2019-05-07 07:00:00
2019-05-07 07:00:00
2019-05-07 06:00:00
2019-05-07 04:00:00
2019-05-07 03:00:00
2019-05-07 02:00:00
2019-05-06 15:00:00
2019-05-06 15:00:00
2019-05-06 09:00:00


Found reply for tweet :1170672009325363200 ,reply with id : 1173122269389787136 ,count is1
Found reply for tweet :1171273529339174913 ,reply with id : 1173098953652363265 ,count is17
Found reply for tweet :1171273529339174913 ,reply with id : 1173081517540511744 ,count is18
Found reply for tweet :1171265121026330624 ,reply with id : 1173073375540858880 ,count is1
Found reply for tweet :1171273529339174913 ,reply with id : 1173071763648892928 ,count is19
Found reply for tweet :1171273529339174913 ,reply with id : 1173049234024615938 ,count is20
Found reply for tweet :1171359045908750337 ,reply with id : 1173016616725962752 ,count is1
Found reply for tweet :1171273529339174913 ,reply with id : 1173014124252606469 ,count is21
Found reply for tweet :1171359045908750337 ,reply with id : 1172973360126447616 ,count is2
Found reply for tweet :1171820366030766080 ,reply with id : 1172949575033925632 ,count is1
Fetching tweets for since id :1170191846221312002 max id is :1172879971909607424
Foun

Found reply for tweet :1171265121026330624 ,reply with id : 1171983287801262080 ,count is12
Found reply for tweet :1171265121026330624 ,reply with id : 1171974500625215488 ,count is13
Found reply for tweet :1171470063112609793 ,reply with id : 1171954615719317504 ,count is3
Found reply for tweet :1171648150366539776 ,reply with id : 1171954240652070913 ,count is2
Found reply for tweet :1171773802080088064 ,reply with id : 1171953279175188482 ,count is3
Found reply for tweet :1171098515344257024 ,reply with id : 1171873660136615936 ,count is7
Found reply for tweet :1171451584481394695 ,reply with id : 1171869210059976704 ,count is4
Found reply for tweet :1171098515344257024 ,reply with id : 1171865551787806720 ,count is8
Found reply for tweet :1171773802080088064 ,reply with id : 1171847017535950849 ,count is4
Found reply for tweet :1171098515344257024 ,reply with id : 1171846077160411137 ,count is9
Found reply for tweet :1171648150366539776 ,reply with id : 1171840037564145665 ,count i

...

Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id

Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id

Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id

Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id

Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id :1170191846221312002 max id is :1170191855914541056
Fetching tweets for since id

In [ ]:
import datetime
import tweepy
import json
import time
import emoji


access_token = "3249597226-fr2Cc6SnoQayvTnc6qCL7mkBDYAEIo1JxgSqJbJ"
access_token_secret = "6SSiNbfdjmLu6Nsh9vdoMEgMlgdl5sOLIBn7TkkfM8Cfg"
consumer_key = "Zp8BVvUHOJKnsH4pgS1Vtxrh1"
consumer_secret = "CnMsDDeR7Yu6KRAMOjPXdjA8Az31OAlHGl0s4YG66SCdbYzYrx"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret) 
api = tweepy.API(auth, wait_on_rate_limit=True) 
original_tweet=0

def convertToObj(tweet):
    obj={}
    obj["poi_name"] = 'narendramodi'
    obj["poi_id"] = 18839785
    obj["verified"] = False
    obj["country"] = "India"
    obj["replied_to_tweet_id"] = tweet.in_reply_to_status_id_str
    obj["replied_to_user_id"] = tweet.in_reply_to_user_id_str
    if tweet.in_reply_to_user_id:
        obj["reply_text"] = tweet.full_text
    obj["tweet_text"] = tweet.full_text
    obj["tweet_lang"] = tweet.lang
    obj["hashtags"] = tweet.entities["hashtags"]
    obj["mentions"] = tweet.entities["user_mentions"]
    obj["id"] = tweet.id
    obj["retweet_count"] = tweet.retweet_count
    obj["favorite_count"] = tweet.favorite_count
    obj["favorited"] = tweet.favorited
    obj["tweet_urls"] = tweet.entities["urls"]
    obj["tweet_emoticons" ] = ''.join(c for c in tweet.full_text if c in emoji.UNICODE_EMOJI)
    obj["tweet_date"] = str(round_to_hour(tweet.created_at))
    return obj

def round_to_hour(dt):
    dt_start_of_hour = dt.replace(minute=0, second=0, microsecond=0)
    dt_half_hour = dt.replace(minute=30, second=0, microsecond=0)

    if dt >= dt_half_hour:
        # round up
        dt = dt_start_of_hour + datetime.timedelta(hours=1)
    else:
        # round down
        dt = dt_start_of_hour

    return dt

for tweet in tweepy.Cursor(api.search, q="#MODIfied100", tweet_mode='extended', count=3000).items(6000):
    if not hasattr(tweet, 'retweeted_status') and not tweet.in_reply_to_status_id and tweet.lang == 'hi':
      original_tweet+=1
      print(original_tweet)
#       print(tweet)
      with open('extra-hash-tags.json', 'a+', encoding="utf8") as f:
        obj=convertToObj(tweet)
        json.dump(obj, f, ensure_ascii=False)
        
time.sleep(1)


print('Original count', original_tweet)